# Naive Bayes Classifier with $k$-mer Encoding

In [2]:
import os
import pickle

# SADR: path to the dataset.
dataset_path = os.path.join("preprocessed_datasets", "dataset_k_mer.pkl")

# SADR: loading training data.
with open(dataset_path, "rb") as f:
    dataset_k_mer = pickle.load(f)

# SADR: getting the training, validation, and testing data.
X_train, y_train = dataset_k_mer["X_train"], dataset_k_mer["y_train"]
X_val, y_val = dataset_k_mer["X_val"], dataset_k_mer["y_val"]
X_test, y_test = dataset_k_mer["X_test"], dataset_k_mer["y_test"]

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"X_val: {X_val.shape}")

X_train: (2560, 459)
X_test: (800, 459)
X_val: (640, 459)


In [3]:
# Import libraries
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Combine train and validation sets
X_combined = np.concatenate((X_train, X_val), axis=0)
y_combined = np.concatenate((y_train, y_val), axis=0)

# Define models to compare
models = {
    "MultinomialNB": MultinomialNB(),
    "BernoulliNB": BernoulliNB(),
    "GaussianNB": GaussianNB()
}

# Set up 5-fold cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Train and evaluate using cross-validation
results = {}
for name, model in models.items():
    # Perform cross-validation
    cv_scores = cross_val_score(model, X_combined, y_combined, 
                               cv=kfold, scoring='f1', n_jobs=-1)
    
    # Store results
    results[name] = {
        "mean_f1": np.mean(cv_scores),
        "std_f1": np.std(cv_scores),
        "all_scores": cv_scores
    }
    
    # Print results
    print(f"----- {name} -----")
    print(f"Mean F1 Score: {results[name]['mean_f1']:.4f} (±{results[name]['std_f1']:.4f})")
    print(f"All F1 Scores: {[f'{score:.4f}' for score in cv_scores]}\n")

# Select best model based on mean validation F1
best_model_name = max(results, key=lambda x: results[x]["mean_f1"])
print(f"\nBest Model: {best_model_name} (Mean CV F1 = {results[best_model_name]['mean_f1']:.4f})")

# Optional: Train the best model on the full combined dataset for final use
best_model = models[best_model_name]
best_model.fit(X_combined, y_combined)

----- MultinomialNB -----
Mean F1 Score: 0.7066 (±0.0269)
All F1 Scores: ['0.7295', '0.7086', '0.7420', '0.6727', '0.6804']

----- BernoulliNB -----
Mean F1 Score: 0.9273 (±0.0135)
All F1 Scores: ['0.9451', '0.9347', '0.9102', '0.9130', '0.9335']

----- GaussianNB -----
Mean F1 Score: 0.9542 (±0.0049)
All F1 Scores: ['0.9519', '0.9521', '0.9600', '0.9472', '0.9596']


Best Model: GaussianNB (Mean CV F1 = 0.9542)


GaussianNB()

In [7]:
# First ensure you have the best model trained on combined data (from previous step)
best_model = models[best_model_name]
best_model.fit(X_combined, y_combined)  # Train on all available data (train + val)

# Now evaluate on test set
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Predict on test set
y_test_pred = best_model.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, pos_label=1)  # Assuming 1 is "human"
recall = recall_score(y_test, y_test_pred, pos_label=1)
f1 = f1_score(y_test, y_test_pred, pos_label=1)

# Print comprehensive results
print(f"\n=== Final Evaluation on Test Set ===")
print(f"Best Model: {best_model_name}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, target_names=["animal", "human"]))




=== Final Evaluation on Test Set ===
Best Model: GaussianNB
Accuracy: 0.9487
Precision: 0.9560
Recall: 0.9444
F1 Score: 0.9502

Classification Report:
              precision    recall  f1-score   support

      animal       0.94      0.95      0.95       386
       human       0.96      0.94      0.95       414

    accuracy                           0.95       800
   macro avg       0.95      0.95      0.95       800
weighted avg       0.95      0.95      0.95       800



In [5]:
# Import required libraries
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, f1_score
import numpy as np

# --- STEP 1: Combine training and validation data ---
X_train_val = np.vstack([X_train, X_val])  # Stack features vertically
y_train_val = np.concatenate([y_train, y_val])  # Concatenate labels

# --- STEP 2: Train GaussianNB (your best model) on full training data ---
print("Training GaussianNB on combined training + validation data...")
model = GaussianNB()
model.fit(X_train_val, y_train_val)

# --- STEP 3: Evaluate on test data (only touched here at the very end) ---
print("\nEvaluating on test set:")
y_test_pred = model.predict(X_test)

# Print comprehensive performance report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, 
                            target_names=["animal", "human"], 
                            digits=4))  # <-- Increased precision

# Print key metrics with 4 decimal places
test_f1 = f1_score(y_test, y_test_pred, pos_label=1)
print(f"\nTest F1 Score (human class): {test_f1:.4f}")
print(f"Test Accuracy: {model.score(X_test, y_test):.4f}")

# --- (Optional) Save the trained model ---
# import pickle
# with open("gaussian_nb_influenza_classifier.pkl", "wb") as f:
#     pickle.dump(model, f)



Training GaussianNB on combined training + validation data...

Evaluating on test set:

Classification Report:
              precision    recall  f1-score   support

      animal     0.9412    0.9534    0.9472       386
       human     0.9560    0.9444    0.9502       414

    accuracy                         0.9487       800
   macro avg     0.9486    0.9489    0.9487       800
weighted avg     0.9488    0.9487    0.9488       800


Test F1 Score (human class): 0.9502
Test Accuracy: 0.9487


In [15]:
# --- STEP 4: Feature Importance Analysis ---
print("\nFeature Importance Analysis:")

# Calculate the absolute difference in means between classes for each feature
mean_diff = np.abs(model.theta_[1] - model.theta_[0])  # Human vs. animal means

# Normalize the differences to get relative importance
feature_importance = mean_diff / np.sum(mean_diff)

# Print feature importance scores (sorted)
print("Feature importance (based on mean differences between classes):")
for idx, importance in sorted(enumerate(feature_importance), key=lambda x: -x[1]):
    print(f"Feature {idx}: {importance:.4f}")


Feature Importance Analysis:
Feature importance (based on mean differences between classes):
Feature 300: 0.0140
Feature 348: 0.0136
Feature 124: 0.0126
Feature 123: 0.0114
Feature 173: 0.0107
Feature 279: 0.0106
Feature 344: 0.0102
Feature 326: 0.0100
Feature 74: 0.0097
Feature 227: 0.0094
Feature 10: 0.0094
Feature 86: 0.0087
Feature 171: 0.0086
Feature 347: 0.0083
Feature 257: 0.0082
Feature 100: 0.0081
Feature 294: 0.0080
Feature 149: 0.0078
Feature 258: 0.0077
Feature 162: 0.0076
Feature 282: 0.0076
Feature 169: 0.0076
Feature 277: 0.0075
Feature 203: 0.0073
Feature 79: 0.0073
Feature 220: 0.0073
Feature 325: 0.0072
Feature 318: 0.0071
Feature 2: 0.0070
Feature 195: 0.0070
Feature 381: 0.0069
Feature 289: 0.0066
Feature 350: 0.0065
Feature 255: 0.0064
Feature 35: 0.0063
Feature 262: 0.0060
Feature 92: 0.0057
Feature 244: 0.0056
Feature 336: 0.0055
Feature 204: 0.0055
Feature 198: 0.0054
Feature 250: 0.0054
Feature 85: 0.0054
Feature 335: 0.0054
Feature 358: 0.0053
Feature 303: 0.

In [16]:
# --- STEP 4: Feature Importance Analysis ---
print("\nFeature Importance Analysis:")

# Calculate absolute difference in means between classes (human vs. animal)
mean_diff = np.abs(model.theta_[1] - model.theta_[0])  # Human vs. animal means
feature_importance = mean_diff / np.sum(mean_diff)  # Normalize

# Get feature names (k-mers) if available
# Example: If you used CountVectorizer, uncomment:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(analyzer='char', ngram_range=(k, k))  # Replace `k` with your k-mer size
# vectorizer.fit(your_sequences)  # Replace with your original data
# feature_names = vectorizer.get_feature_names_out()

# If you have a predefined list of k-mers, assign it here:
# feature_names = ["ATG", "GCA", "TAA", ...]  # Replace with your k-mers

# Print sorted feature importance (with k-mers if available)
print("Top 10 most important features:")
for idx, importance in sorted(enumerate(feature_importance), key=lambda x: -x[1])[:10]:
    if 'feature_names' in locals():
        print(f"K-mer '{feature_names[idx]}': {importance:.4f}")
    else:
        print(f"Feature {idx}: {importance:.4f}")


Feature Importance Analysis:
Top 10 most important features:
Feature 300: 0.0140
Feature 348: 0.0136
Feature 124: 0.0126
Feature 123: 0.0114
Feature 173: 0.0107
Feature 279: 0.0106
Feature 344: 0.0102
Feature 326: 0.0100
Feature 74: 0.0097
Feature 227: 0.0094
